In [1]:
import pandas as pd
import numpy as np

In [29]:
AT1 = ['4YAY','4ZUD','6DO1','6OS0','6OS1','6OS2']
arr = ['6TKO','6U1N','6UP7','6PWC']

In [2]:
hyd_aci = pd.read_csv('/home/kestr/AT1/data/hydrogen_contacts.csv')
repulsion = pd.read_csv('/home/kestr/AT1/data/charge_repulsion.csv')
other_contacts = pd.read_csv('/home/kestr/AT1/data/contacts_ion_and_hydrophobic.csv')
total = pd.read_csv('/home/kestr/AT1/data/model_res_dict.csv')
aligned_cutted_arr_path = '../pdb/2_aligned/cutted_arr'

In [6]:
hyd_aci=hyd_aci[['gpcr','arr','chain','Cluster_ID','res']]
hyd_aci['type'] = 'HY'
repulsion=repulsion[['gpcr','arr','chain','Cluster_ID','res']]
repulsion['type'] = 'RE'
contacts = pd.concat([other_contacts, hyd_aci, repulsion])
contacts_R = contacts[contacts.chain == 'R']
contacts_A = contacts[contacts.chain == 'A']
contacts_A = contacts_A.merge(total,left_on=['arr','chain','res'],right_on=['pdb_code','chain_id','residue_number'])
contacts_R = contacts_R.merge(total,left_on=['gpcr','chain','res'],right_on=['pdb_code','chain_id','residue_number'])

/tmp/ipykernel_174062/511699616.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hyd_aci['type'] = 'HY'


In [57]:
# Отрисовка кластеров контактов
script = f'''
    reinitialize
    preset.publication(selection='all')
    '''

for AT1_code in AT1:
    for arr_code in arr:
        script += f'''
    load ../{aligned_cutted_arr_path}/{AT1_code}_{arr_code}.pdb, {AT1_code}_{arr_code}
    alter chain T, chain='R'
    util.cbag(selection='chain R')
    util.cbak(selection='chain A')
    hide all
    '''
        con_R=contacts_R[contacts_R.gpcr==AT1_code][contacts_R.arr==arr_code]
        con_A=contacts_A[contacts_A.gpcr==AT1_code][contacts_A.arr==arr_code]
        for BW in con_R.b_factor.unique():
            cont_R_BW = con_R[con_R.b_factor==BW]
            for clu in cont_R_BW['Cluster_ID'].unique():
                cont_R_BW_clu = cont_R_BW[cont_R_BW.Cluster_ID==clu]
                for typec in cont_R_BW_clu['type'].unique():
                    arr_reses = con_A[con_A.Cluster_ID==clu][con_A.type==typec]
                    cont_R_BW_clu_type = cont_R_BW_clu[cont_R_BW_clu.type==typec]
                    flag = 0
                    for arr_res in arr_reses.res.unique():
                        gpcr_res = (cont_R_BW_clu_type[cont_R_BW_clu_type.pdb_code == AT1_code].residue_number.iloc[0])
                        gpcr_res_name = (cont_R_BW_clu_type[cont_R_BW_clu_type.pdb_code == AT1_code].residue_name.iloc[0])
                        if(flag == 0):
                            script+=f'''select init_res, {AT1_code}_{arr_code} and chain R and resi {gpcr_res}\n'''
                            flag = 1
                        script+=f'''select init_res, {AT1_code}_{arr_code} and chain A and resi {arr_res} or init_res\n'''
                    script+=f'''
                        show sticks, init_res
                        h_add (ele n+o+s+se) and init_res
                        contacts {AT1_code}_{arr_code} and chain A and init_res, {AT1_code}_{arr_code} and chain R and init_res, result="contacts_{AT1_code}_{arr_code}", cutoff=3.6, bigcutoff=3.6
                        label n. N & ! chain R and init_res, resi+resn
                        label n. N & chain R and init_res, resi+resn+' %1.2f'%b
                        set label_position,(0,1.5,4)
                        show cartoon, {AT1_code}_{arr_code}
                        set cartoon_transparency, 0.5
                        orient init_res
                        viewport 1024,768
                        bg_color white
                        set label_size, 25
                        set label_font_id, 7
                        move z,-1
                        ray
                        png ../../images/res_clusters/{typec}/{BW}_{AT1_code}_{arr_code}_{clu}.png
                        hide all


                        '''

with open(f'../scripts/3_images/images_clu.pml', 'w') as f:
            f.write(script)

/tmp/ipykernel_174062/741765378.py:16: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  con_R=contacts_R[contacts_R.gpcr==AT1_code][contacts_R.arr==arr_code]
/tmp/ipykernel_174062/741765378.py:17: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  con_A=contacts_A[contacts_A.gpcr==AT1_code][contacts_A.arr==arr_code]
/tmp/ipykernel_174062/741765378.py:23: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  arr_reses = con_A[con_A.Cluster_ID==clu][con_A.type==typec]
